In [2]:
import form
import re
f = form.open("form -l",keep_log=100)

In [25]:
k = [[" 1 ", " 1 ", " 1 ", " 1 "], 
     [" 1 ", " 1 ", " i_", "-i_"], 
     [" 1 ", "-i_", " 1 ", " i_"], 
     [" 1 ", " i_", "-i_", " 1 "]]


n = [[" 1       ", "cx(i1,i2)", "cy(i1,i2)", "cz(i1,i2)"], 
     ["cx(i1,i2)", " 1       ", "cz(i1,i2)", "cy(i1,i2)"], 
     ["cy(i1,i2)", "cz(i1,i2)", " 1       ", "cx(i1,i2)"], 
     ["cz(i1,i2)", "cy(i1,i2)", "cx(i1,i2)", " 1       "]]

c = ["1","cx","cy","cz"]
s = ["1","sx","sy","sz"]
rules = '\n'.join('id ifmatch->endarg '+c[i]+'(i1?,i2?)*'+s[j]+'(i1?,i2?) = '+
                  k[i][j]+'*'+str(n[j][i])+';' 
       for i in range(1,4) for j in range(1,4))
#for i in range(4):
#    print()
#    for j in range(4):
#        print('id m'+str(i)+'*m'+str(j)+' = '\
#              +k[i][j]+'*'+str(n[j][i])+';')
print(rules)

id ifmatch->endarg cx(i1?,i2?)*sx(i1?,i2?) =  1 * 1       ;
id ifmatch->endarg cx(i1?,i2?)*sy(i1?,i2?) =  i_*cz(i1,i2);
id ifmatch->endarg cx(i1?,i2?)*sz(i1?,i2?) = -i_*cy(i1,i2);
id ifmatch->endarg cy(i1?,i2?)*sx(i1?,i2?) = -i_*cz(i1,i2);
id ifmatch->endarg cy(i1?,i2?)*sy(i1?,i2?) =  1 * 1       ;
id ifmatch->endarg cy(i1?,i2?)*sz(i1?,i2?) =  i_*cx(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sx(i1?,i2?) =  i_*cy(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sy(i1?,i2?) = -i_*cx(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sz(i1?,i2?) =  1 * 1       ;


In [44]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;

Function S,SS,Comm;
cfun koef;
*polyfun koef;

auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

Local C = Comm(2*cx(3,1)*cx(3,2),cy(1,1)*cy(2,1));

*--- overlapping ---
argument Comm;
    multiply S(1,1,1);
    repeat id S(k1?,k2?,t?)*cc?CC(i1?,i2?) = S(max_(k1,i1),max_(k2,i2),t*cc(i1,i2));
endargument;
id Comm(a?,b?) = a*b;
id S(?ss1)*S(?ss2) = Comm(S(?ss1),S(?ss2));
id Comm(S(n1?,k1?,a1?),S(n2?,k2?,a2?)) = 
                  Comm(S(k1,        a1),S(k2,        a2))  + 
    sum_(i,1,n2-1,Comm(S(k1,SS(i,1)*a1),S(k2,        a2))) +
    sum_(i,1,n1-1,Comm(S(k1,        a1),S(k2,SS(i,1)*a2)));
argument Comm; argument S,2;
    repeat id SS(i?,t?)*cc?CC(j?,k?) = SS(i,t*cc(i+j,k));
    id SS(i?,a?) = a;
endargument;endargument;
id Comm(S(n1?,a1?),S(n2?,a2?)) = 
                  Comm(        a1,        a2) + 
    sum_(i,1,n2-1,Comm(SS(i,1)*a1,        a2)) +
    sum_(i,1,n1-1,Comm(        a1,SS(i,1)*a2));
argument Comm;
    repeat id SS(i?,t?)*cc?CC(k?,j?) = SS(i,t*cc(k,i+j));
    id SS(i?,a?) = a;
endargument;

*--- multiply ---
id Comm(a?,b?) = S(a)*b-S(b)*a;
label repeat;
    id ifnomatch->endrepeat S(a?)*cc?CC[n](i1?,i2?) = S(a*SC[n](i1,i2));
    argument S;
*        id cc?CC(i1?,i2?)*sc?SC(i1?,i2?) = 1;
'''+rules+
'''
        id sc?SC[n?](i1?,i2?) = CC[n](i1,i2);
        label endarg;
    endargument;
goto repeat;
label endrepeat;
id S(a?) = a;
.sort

*--- trim ---
if(match(cc?CC(i?,j?)));
    multiply S(1000000,1000000,1);
    repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(min_(n1,i1),min_(n2,i2),t*cc(i1,i2));
    id ifmatch->endif S(1,1,t?) = t;
        id S(n1?,n2?,t?) = S(n1-1,n2-1,1)*t;
        repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(n1,n2,t*cc(i1-n1,i2-n2));
        id S(n1?,n2?,t?) = t;
    label endif;
endif;

Print;
.sort
''')
print(re.sub(r'([+-])','\n\g<1>',f.read('C')))
#f.read('C')

4*cx(1,1)*cy(2,2)*cz(1,2)*i_
+4*cx(1,2)*cy(2,1)*cz(1,1)*i_
+4*cx(2,1)*cy(1,2)*cz(2,2)*i_
+4*cx(2,2)*cy(1,1)*cz(2,1)*i_


In [42]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
Fun Comm;
auto sym a,b;

L L = Comm(a1+a2+a3,b1+b2);

*--- expand ---
splitarg Comm,1;
repeat;
id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
endrepeat;
splitarg Comm,2;
repeat;
id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
endrepeat;
.sort
''')
f.read('L')

'Comm(a1,b1)+Comm(a1,b2)+Comm(a2,b1)+Comm(a2,b2)+Comm(a3,b1)+Comm(a3,b2)'

In [47]:
procedures = '''
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;

Function S,SS,Comm;

auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

*--- expand ---
#procedure expand
splitarg Comm,1;
repeat;
id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
endrepeat;
splitarg Comm,2;
repeat;
id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
endrepeat;
#endprocedure

*--- overlapping ---
#procedure overlapping
argument Comm;
    multiply S(1,1,1);
    repeat id S(k1?,k2?,t?)*cc?CC(i1?,i2?) = S(max_(k1,i1),max_(k2,i2),t*cc(i1,i2));
endargument;
id Comm(a?,b?) = a*b;
id S(?ss1)*S(?ss2) = Comm(S(?ss1),S(?ss2));
id Comm(S(n1?,k1?,a1?),S(n2?,k2?,a2?)) = 
                  Comm(S(k1,        a1),S(k2,        a2))  + 
    sum_(i,1,n2-1,Comm(S(k1,SS(i,1)*a1),S(k2,        a2))) +
    sum_(i,1,n1-1,Comm(S(k1,        a1),S(k2,SS(i,1)*a2)));
argument Comm; argument S,2;
    repeat id SS(i?,t?)*cc?CC(j?,k?) = SS(i,t*cc(i+j,k));
    id SS(i?,a?) = a;
endargument;endargument;
id Comm(S(n1?,a1?),S(n2?,a2?)) = 
                  Comm(        a1,        a2) + 
    sum_(i,1,n2-1,Comm(SS(i,1)*a1,        a2)) +
    sum_(i,1,n1-1,Comm(        a1,SS(i,1)*a2));
argument Comm;
    repeat id SS(i?,t?)*cc?CC(k?,j?) = SS(i,t*cc(k,i+j));
    id SS(i?,a?) = a;
endargument;
#endprocedure

*--- multiply ---
#procedure multiply
*id Comm(a?,b?) = S(a)*b-S(b)*a;
label repeat;
    id ifnomatch->endrepeat S(a?)*cc?CC[n](i1?,i2?) = S(a*SC[n](i1,i2));
    argument S;
*        id cc?CC(i1?,i2?)*sc?SC(i1?,i2?) = 1;
'''+rules+\
'''
        id sc?SC[n?](i1?,i2?) = CC[n](i1,i2);
        label endarg;
    endargument;
goto repeat;
label endrepeat;
id S(a?) = a;
*.sort
#endprocedure

*--- trim ---
#procedure trim
if(match(cc?CC(i?,j?)));
    multiply S(1000000,1000000,1);
    repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(min_(n1,i1),min_(n2,i2),t*cc(i1,i2));
    id ifmatch->endif S(1,1,t?) = t;
        id S(n1?,n2?,t?) = S(n1-1,n2-1,1)*t;
        repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(n1,n2,t*cc(i1-n1,i2-n2));
        id S(n1?,n2?,t?) = t;
    label endif;
endif;
#endprocedure
'''

In [50]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
'''+procedures+
'''

Local H  = cx(1,1)*cx(1,2) + cy(1,1)*cy(2,1);
Local N1 = cx(1,1)*cx(1,2) - cy(1,1)*cy(2,1);
Local N2 = Comm(H,N1);

#call expand
#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

Print;
.sort
''')
print(re.sub(r'([+-])','\n\g<1>',f.read('N2')))


-4*cx(1,1)*cy(2,2)*cz(1,2)*i_
-4*cx(1,2)*cy(2,1)*cz(1,1)*i_
-4*cx(2,1)*cy(1,2)*cz(2,2)*i_
-4*cx(2,2)*cy(1,1)*cz(2,1)*i_


In [51]:
f.write('''
Local N3 = Comm(H,N2);

#call expand
#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

Print;
.sort
''')
print(re.sub(r'([+-])','\n\g<1>',f.read('N3')))

32*cx(1,1)*cx(1,2)
-16*cx(1,1)*cx(1,3)*cy(1,2)*cy(2,2)
+16*cx(1,1)*cx(2,1)*cz(1,2)*cz(2,2)
+16*cx(1,1)*cx(2,3)*cz(1,2)*cz(2,2)
+16*cx(1,2)*cx(2,2)*cz(1,1)*cz(2,1)
+16*cx(1,3)*cx(2,1)*cz(1,2)*cz(2,2)
+16*cx(2,1)*cx(2,2)*cy(1,1)*cy(3,1)
+16*cx(2,1)*cx(2,2)*cy(1,2)*cy(3,2)
-16*cx(2,1)*cx(2,3)*cy(1,2)*cy(2,2)
-16*cy(1,1)*cy(1,2)*cz(2,1)*cz(2,2)
-32*cy(1,1)*cy(2,1)
-16*cy(1,1)*cy(3,2)*cz(2,1)*cz(2,2)
-16*cy(1,2)*cy(3,1)*cz(2,1)*cz(2,2)
-16*cy(2,1)*cy(2,2)*cz(1,1)*cz(1,2)


In [52]:
f.write('''
Local N4 = Comm(H,N3);

#call expand
#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

Print;
.sort
''')
print(re.sub(r'([+-])','\n\g<1>',f.read('N4')))


-96*cx(1,1)*cx(1,3)*cx(2,1)*cy(1,2)*cz(2,2)*i_
-96*cx(1,1)*cx(1,3)*cx(2,3)*cy(1,2)*cz(2,2)*i_
+64*cx(1,1)*cx(2,1)*cx(2,2)*cy(3,2)*cz(1,2)*i_
-96*cx(1,1)*cx(2,1)*cx(2,3)*cy(2,2)*cz(1,2)*i_
+64*cx(1,1)*cx(2,2)*cx(2,3)*cy(3,2)*cz(1,2)*i_
+64*cx(1,1)*cy(1,2)*cy(2,2)*cy(2,3)*cz(1,3)*i_
-64*cx(1,1)*cy(1,3)*cz(1,2)*cz(2,2)*cz(2,3)*i_
-320*cx(1,1)*cy(2,2)*cz(1,2)*i_
-64*cx(1,1)*cy(3,1)*cz(1,2)*cz(2,1)*cz(2,2)*i_
-64*cx(1,1)*cy(3,3)*cz(1,2)*cz(2,2)*cz(2,3)*i_
+64*cx(1,2)*cx(2,1)*cx(2,2)*cy(3,1)*cz(1,1)*i_
-320*cx(1,2)*cy(2,1)*cz(1,1)*i_
-64*cx(1,2)*cy(3,2)*cz(1,1)*cz(2,1)*cz(2,2)*i_
+64*cx(1,3)*cx(2,1)*cx(2,2)*cy(3,2)*cz(1,2)*i_
-96*cx(1,3)*cx(2,1)*cx(2,3)*cy(2,2)*cz(1,2)*i_
-64*cx(1,3)*cy(1,1)*cz(1,2)*cz(2,1)*cz(2,2)*i_
+64*cx(1,3)*cy(1,2)*cy(2,1)*cy(2,2)*cz(1,1)*i_
-64*cx(1,3)*cy(3,1)*cz(1,2)*cz(2,1)*cz(2,2)*i_
+64*cx(2,1)*cx(2,2)*cx(3,1)*cy(1,2)*cz(3,2)*i_
+64*cx(2,1)*cx(2,2)*cx(3,2)*cy(1,1)*cz(3,1)*i_
+64*cx(2,1)*cx(2,2)*cx(3,3)*cy(1,2)*cz(3,2)*i_
-96*cx(2,1)*cy(1,1)*cy(1,2)*cy(3,1)*cz(2,2